# Tareas

## Tarea 1

Detección de colores de la ropa y de si es hombre o mujer con un modelo (enrealidad 2) entrenado por nosotros

### Importaciones

In [8]:
import os
import numpy as np
from skimage import io, color
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import kagglehub
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPUs detectadas:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.16.2
GPUs detectadas: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Definición de datasets

In [2]:
dataset_Gender = "DatabaseGender"
dataset_emotions = kagglehub.dataset_download("jonathanoheix/face-expression-recognition-dataset")

### Cargar datasets

In [3]:
def load_dataset(path, image_size=(48, 48)):
    X, y = [], []
    labels = os.listdir(path)
    for label in labels:
        folder = os.path.join(path, label)
        for file in os.listdir(folder):
            img_path = os.path.join(folder, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, image_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

X_gender, y_gender = load_dataset(dataset_Gender, (64,64))
X_emotion, y_emotion = load_dataset(
    os.path.join(dataset_emotions, "images/train"), (48,48)
)



### Normalizar y Etiquetar

In [4]:

X_gender = X_gender / 255.0
X_emotion = X_emotion / 255.0

le_gender = LabelEncoder()
y_gender = le_gender.fit_transform(y_gender)

le_emotion = LabelEncoder()
y_emotion = le_emotion.fit_transform(y_emotion)


### Crear modelo

In [9]:
def build_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

### Aplicar kfold y Entrenar

In [10]:
early_stop = EarlyStopping(
    monitor='val_loss',   # monitorea la pérdida de validación
    patience=3,           # se detiene si no mejora durante 3 epochs
    restore_best_weights=True  # restaura los pesos del mejor epoch
)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

X = X_emotion  # o X_gender
y = y_emotion  # o y_gender
X = X.astype("float32") / 255.0
fold = 1
for train_idx, val_idx in kf.split(X):
    print(f"\n📂 Fold {fold}")
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model = build_cnn(X.shape[1:], len(np.unique(y)))
    model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stop], batch_size=32)
    
    fold += 1


📂 Fold 1


/opt/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.2432 - loss: 1.8311 - val_accuracy: 0.2402 - val_loss: 1.8155
Epoch 2/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.2484 - loss: 1.8187 - val_accuracy: 0.2402 - val_loss: 1.8144
Epoch 3/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.2594 - loss: 1.7911 - val_accuracy: 0.2808 - val_loss: 1.7427
Epoch 4/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.2866 - loss: 1.7596 - val_accuracy: 0.3108 - val_loss: 1.7007
Epoch 5/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.2991 - loss: 1.7466 - val_accuracy: 0.3282 - val_loss: 1.6730
Epoch 6/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.3171 - loss: 1.7244 - val_accuracy: 0.3414 - val_loss: 1.6621
Epoch 7/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.3243 - loss: 1.7085 - val_accuracy: 0.3402 - val_loss: 1.6552
Epoch 8/20
721/721 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.3307 - loss: 1.7002 - 

### Guardar Modelo

In [ ]:
model.save("cnn_emotion_model.h5")

NameError: name 'model' is not defined

### Carga de Modelo
El paso de guardar el modelo anterior no es necesario, pero en mi caso si ya que en macOS la camara tiene la tendencia de crashear el kernel de python y es necesario recargar todo y si tuviese que reentrenar el modelo en cada ocasión no podría hacer una defensa a tiempo.

In [11]:
model = load_model("cnn_emotion_model.h5")

### Carga de filtros


In [ ]:
def filtro_happy(frame):
    # Aumentar brillo
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    v = cv2.add(v, 50)  # subir brillo
    v = np.clip(v, 0, 255)
    final_hsv = cv2.merge((h,s,v))
    return cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

def filtro_sad(frame):
    # Tonos azules (enfriar imagen)
    frame = cv2.addWeighted(frame, 0.7, np.zeros_like(frame), 0.3, 0)
    frame[:,:,0] = cv2.add(frame[:,:,0], 50)  # azul más intenso
    return frame

def filtro_angry(frame):
    # Tonos rojos
    frame = cv2.addWeighted(frame, 0.7, np.zeros_like(frame), 0.3, 0)
    frame[:,:,2] = cv2.add(frame[:,:,2], 50)  # rojo más intenso
    return frame


### Resolución

In [ ]:
labels = le_emotion.classes_  # ej. ['angry', 'happy', 'sad', ...]

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (48,48)) / 255.0
    img_resized = np.expand_dims(img_resized, axis=0)

    preds = model.predict(img_resized)
    label = labels[np.argmax(preds)]
    confidence = np.max(preds)

    # Aplicar filtro según emoción
    if label == "happy":
        frame = filtro_happy(frame)
    elif label == "sad":
        frame = filtro_sad(frame)
    elif label == "angry":
        frame = filtro_angry(frame)
    # etc.

    cv2.putText(frame, f"{label} ({confidence*100:.1f}%)", (10,30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Emotion Filters", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━

: 